In [45]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt


In [46]:
def payoff_calc(S_ti, K):
    """
    This function calculates future payoff of the asian option based on arithmetic average of the price path
    
    INPUT:
        price_array (numpy.ndarray): A one-dimensional array of stock final prices
        K (float): Exercise price of the option
    
    OUTPUT:
        (numpy.ndarray): A one dimensional array of payoffs for different prices
    """
    
    payoff = np.maximum(0, np.mean(S_ti) - K)
    return payoff

In [47]:
def pv_calc(FV, r, T):
    """
    Calculates present value of an amount of money in future.
    
    INPUT:
        FV (float): Future value of money
        r (float): Risk neutral interest rate
        T (float): Period of time
    
    OUTPUT:
        (float): Present value of FV
    """
    
    return FV * np.exp(-r * T)

In [48]:
def ordinary_mc_sim(no_of_paths, n_steps, S_0, T, r, sigma, x_price):
    """
    Conducts MC simulation,
    
    INPUT:
        no_of_paths (int): Number of samples in simulation
        n_steps (int): Number of price step we aim to simulate in each path
        S_0 (float): Underlying asset price at time zero
        T (float): Time period of option contract
        r (float): Risk-netural interest rate
        sigma (float): Volatility in the environment
        x_price (float): Exercise price of the option
        
    OUTPUT:
        (Numpy.ndarray): A one-dimensional array of present value of simulated payoffs
    """
    dt = T / n_steps
    present_payoffs = np.zeros(no_of_paths)
    epsilon = np.random.normal(0, 1, (no_of_paths, n_steps))
    
    for k in range(no_of_paths):
        price_steps = np.zeros(n_steps)
        price_steps[0] = S_0
        for i in range(1, n_steps):
            price_steps[i] = price_steps[i-1] * np.exp((r - sigma ** 2 / 2) * dt + sigma * epsilon[k, i] * np.sqrt(dt))
        present_payoffs[k] = pv_calc(payoff_calc(price_steps, x_price), r, T)
    return(present_payoffs)

In [49]:
def sim_iterator(max_sample, n_steps, S_0, T, r, sigma, x_price, *,method):
    """
    Iterates simulation with different sample sizes (form 10 to a maximum size with steps of 10)
    
    INPUT:
        max_sample (int): Maximum sample size for the iteration of simulations
        n_steps (int): Number of price step we aim to simulate in each path
        S_0 (float): Underlying asset price at time zero
        T (float): Time period of option contract
        r (float): Risk-netural interest rate
        sigma (float): Volatility in the environment
        x_price (float): Exercise price of the option
        method (string): 'sobol', 'halton' or 'ordinary'
    
    OUTPUT:
        (numpy.ndarray): confidence intervals of the simulations
        (numpy.ndarray): price estimations of the simulations
    """
    
    mean_pv_payoffs = np.zeros(int(np.ceil(max_sample / 10)))

    for n_sample in range(10, max_sample + 1, 10):
        present_payoffs = ordinary_mc_sim(n_sample, n_steps, S_0, T, r, sigma, x_price)
        mean_pv_payoffs[int(n_sample/10 - 1)] = np.mean(present_payoffs)

    
    return mean_pv_payoffs

In [50]:
r = 0.1
sigma = 0.2
T = 2
n_steps = 20
dt = T / n_steps
S_0 = 50
x_price = 40

max_sample = 7000

present_payoffs_1 = ordinary_mc_sim(1, n_steps, S_0, T, r, sigma, x_price)
present_payoffs_1
present_payoffs_2 = ordinary_mc_sim(2, n_steps, S_0, T, r, sigma, x_price)
present_payoffs_2

array([11.82600633, 12.10386608])